# Calibration Notebook
This notebook helps you generate a pickle file, defining the deformation matrices needed for a correct pose estimation.

You need to record a video, filming a calibration checkboard from various angles and various distances.

Then define the parameters needed for calibration

In [4]:
FRAMERATE = 50 #Only select one in every 100 frames. Less frames is better but more computationally heavy.
video_path = "/mnt/2To/jupyter_data/PdS_LC/program/automated-paper-tracking/calibration/calibr_26K_cam1.MP4" #Path of the video
camera_id = 1

In [21]:
import cv2
import os
from matplotlib import pyplot as plt
import numpy as np
import glob
from dt_apriltags import Detector
from tqdm import tqdm



def extract_frames(video_path):
    # Get the video filename without extension and the directory
    video_name = os.path.basename(video_path)
    video_name_without_ext = os.path.splitext(video_name)[0]
    output_dir = os.path.join(os.path.dirname(video_path), video_name_without_ext)
    
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Open the video file
    video_capture = cv2.VideoCapture(video_path)
    
    # Check if the video opened successfully
    if not video_capture.isOpened():
        print(f"Error: Could not open video {video_path}")
        return
    
    frame_count = 0
    success, frame = video_capture.read()
    
    # Read and save frames
    while success:
        frame_filename = os.path.join(output_dir, f"frame_{frame_count:04d}.jpg")
        cv2.imwrite(frame_filename, frame)
        success, frame = video_capture.read()
        frame_count += 1
    
    video_capture.release()
    print(f"Extracted {frame_count} frames to {output_dir}")
    return video_name_without_ext

# Example usage

output_dir = extract_frames(video_path)


Extracted 14948 frames to /mnt/2To/jupyter_data/PdS_LC/program/automated-paper-tracking/calibration/calibr_26K_cam1


In [2]:
output_dir = '/mnt/2To/jupyter_data/PdS_LC/program/automated-paper-tracking/calibration/calibr_26K_cam1'

In [5]:
def calibrateCamera():
	print("Calibrating ...")

	images = glob.glob(f'{video_path.replace(".MP4","/")}*.jpg') 
	criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
	print(images)
	# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
	objp = np.zeros((9 * 7, 3), np.float32)
	objp[:, :2] = np.mgrid[0:9, 0:7].T.reshape(-1, 2)

	# Arrays to store object points and image points from all the images.
	world_pts = []  # 3d point in real world space
	img_pts = []  # 2d points in image plane.
	gray = None
	for img in tqdm(images[::FRAMERATE]):
		img = cv2.imread(img)
		gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
		
		# Find the chess board corners
		ret, corners = cv2.findChessboardCorners(gray, (9,7), None)
		# If found, add object points, image points (after refining them)
		if ret:
			world_pts.append(objp)
			corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
			img_pts.append(corners2)

			# Draw and display the corners
			img = cv2.drawChessboardCorners(img, (9,7), corners2, ret)
			i = 1
			d = f"img {i}"
	rms, K, dist, rvecs, tvecs = cv2.calibrateCamera(world_pts, img_pts, gray.shape[::-1], None, None)

	h, w = img.shape[:2]
	new_K, roi = cv2.getOptimalNewCameraMatrix(K, dist, (w, h), 1, (w, h))

	print("Intrisincs :", K)
	print("New intrisincs :", new_K)
	print("Dist coef :", dist)
	print("The overall RMS re-projection error :", rms)

	return [K, dist, new_K, roi]

calib_param = calibrateCamera()

Calibrating ...
['/mnt/2To/jupyter_data/PdS_LC/program/automated-paper-tracking/calibration/calibr_26K_cam1/frame_6043.jpg', '/mnt/2To/jupyter_data/PdS_LC/program/automated-paper-tracking/calibration/calibr_26K_cam1/frame_6977.jpg', '/mnt/2To/jupyter_data/PdS_LC/program/automated-paper-tracking/calibration/calibr_26K_cam1/frame_6102.jpg', '/mnt/2To/jupyter_data/PdS_LC/program/automated-paper-tracking/calibration/calibr_26K_cam1/frame_8493.jpg', '/mnt/2To/jupyter_data/PdS_LC/program/automated-paper-tracking/calibration/calibr_26K_cam1/frame_14654.jpg', '/mnt/2To/jupyter_data/PdS_LC/program/automated-paper-tracking/calibration/calibr_26K_cam1/frame_9639.jpg', '/mnt/2To/jupyter_data/PdS_LC/program/automated-paper-tracking/calibration/calibr_26K_cam1/frame_8794.jpg', '/mnt/2To/jupyter_data/PdS_LC/program/automated-paper-tracking/calibration/calibr_26K_cam1/frame_14680.jpg', '/mnt/2To/jupyter_data/PdS_LC/program/automated-paper-tracking/calibration/calibr_26K_cam1/frame_13464.jpg', '/mnt/2T

  1%|          | 2/299 [00:02<05:26,  1.10s/it]

(2704, 1520)
(2704, 1520)


  1%|▏         | 4/299 [00:02<02:12,  2.22it/s]

(2704, 1520)
(2704, 1520)


...

(2704, 1520)
(2704, 1520)


100%|█████████▉| 298/299 [04:53<00:01,  1.10s/it]

(2704, 1520)


100%|██████████| 299/299 [04:55<00:00,  1.01it/s]

(2704, 1520)


Save parameters as pickle file

In [10]:
import pickle
with open(video_path.replace('.MP4','.pkl'), 'wb') as f:
    pickle.dump(calib_param, f)